# Benchmark of various outlier detection models

### The models are evaluaed on ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are splitted 60% for training and 40% for testing.

**[PyOD](https://github.com/yzhao062/Pyod)** is a comprehensive **Python toolkit** to **identify outlying objects** in 
multivariate data with both unsupervised and supervised approaches.


  1. Linear Models for Outlier Detection:
     1. **PCA: Principal Component Analysis** use the sum of
       weighted projected distances to the eigenvector hyperplane 
       as the outlier outlier scores) [10]
     2. **MCD: Minimum Covariance Determinant** (use the mahalanobis distances 
       as the outlier scores) [11, 12]
     3. **One-Class Support Vector Machines** [3]
     
  2. Proximity-Based Outlier Detection Models:
     1. **LOF: Local Outlier Factor** [1]
     2. **CBLOF: Clustering-Based Local Outlier Factor** [15]
     3. **kNN: k Nearest Neighbors** (use the distance to the kth nearest 
     neighbor as the outlier score)
     4. **Average kNN** Outlier Detection (use the average distance to k 
     nearest neighbors as the outlier score)
     5. **Median kNN** Outlier Detection (use the median distance to k nearest 
     neighbors as the outlier score)
     6. **HBOS: Histogram-based Outlier Score** [5]
     
  3. Probabilistic Models for Outlier Detection:
     1. **ABOD: Angle-Based Outlier Detection** [7]
     2. **FastABOD: Fast Angle-Based Outlier Detection using approximation** [7]
  
  4. Outlier Ensembles and Combination Frameworks
     1. **Isolation Forest** [2]
     2. **Feature Bagging** [9]

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [2]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          check_estimator=False,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction,
                                       random_state=random_state),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.171s
Cluster-based Local Outlier Factor ROC:0.778, precision @ rank n:0.5, execution time: 0.0344s
Feature Bagging ROC:0.7736, precision @ rank n:0.5, execution time: 0.5254s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.2122s
Isolation Forest ROC:0.8217, precision @ rank n:0.5, execution time: 0.2036s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.0853s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0702s
Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 0.5374s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.0481s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0572s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD

One-class SVM (OCSVM) ROC:0.4808, precision @ rank n:0.0, execution time: 1.5436s
Principal Component Analysis (PCA) ROC:0.5016, precision @ rank n:0.0, execution time: 0.0537s

... Processing pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6957, precision @ rank n:0.1127, execution time: 1.8649s
Cluster-based Local Outlier Factor ROC:0.9329, precision @ rank n:0.2394, execution time: 0.0669s
Feature Bagging ROC:0.4277, precision @ rank n:0.0986, execution time: 4.0121s
Histogram-base Outlier Detection (HBOS) ROC:0.9322, precision @ rank n:0.3099, execution time: 0.1376s
Isolation Forest ROC:0.9756, precision @ rank n:0.3944, execution time: 0.5828s
K Nearest Neighbors (KNN) ROC:0.7694, precision @ rank n:0.1549, execution time: 0.5576s
Local Outlier Factor (LOF) ROC:0.4056, precision @ rank n:0.0986, execution time: 0.5517s
Minimum Covariance Determinant (MCD) ROC:0.8413, precision @ rank n:0.0986, execution time: 1.609s
One-class SVM (OCSVM) ROC:0.9376, precision @ rank n

In [3]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.171,0.0344,0.5254,0.2122,0.2036,0.0853,0.0702,0.5374,0.0481,0.0572
0,cardio,1831,21,9.6122,0.5818,0.031,0.909,0.0469,0.2626,0.1524,0.0898,0.4314,0.0922,0.004
0,glass,214,9,4.2056,0.0681,0.0192,0.0156,0.0156,0.1253,0.0091,0.002,0.0356,0,0
0,ionosphere,351,33,35.8974,0.0937,0.0156,0.0625,0.0313,0.16,0.0157,0.0156,0.0468,0,0
0,letter,1600,32,6.25,0.4973,0.0156,0.7252,0.0625,0.2632,0.1263,0.0943,0.9454,0.0762,0.004
0,lympho,148,18,4.0541,0.0401,0.017,0.023,0.0071,0.1614,0.005,0.002,0.031,0.002,0.002
0,mnist,7603,100,9.2069,7.7986,0.067,42.0965,0.9069,1.4098,5.7465,5.7854,2.0246,4.2719,0.125
0,musk,3062,166,3.1679,2.2028,0.028,10.1455,0.5653,0.8163,1.5487,1.466,8.3801,1.3012,0.1484
0,optdigits,5216,64,2.8758,2.8534,0.025,10.828,0.4581,0.658,1.7452,1.9845,1.2785,1.5436,0.0537
0,pendigits,6870,16,2.2707,1.8649,0.0669,4.0121,0.1376,0.5828,0.5576,0.5517,1.609,0.8713,0.005


Analyze the performance of ROC and Precision @ n

In [4]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.778,0.7736,0.8511,0.8217,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5668,0.8987,0.5667,0.8102,0.8726,0.7252,0.5313,0.7966,0.9055,0.9237
0,glass,214,9,4.2056,0.7605,0.7457,0.758,0.6346,0.5556,0.8198,0.8395,0.7728,0.5506,0.5506
0,ionosphere,351,33,35.8974,0.92,0.812,0.9004,0.6005,0.8587,0.9378,0.9063,0.9513,0.8497,0.8025
0,letter,1600,32,6.25,0.8992,0.5905,0.8938,0.6328,0.6445,0.8972,0.8821,0.8766,0.6071,0.5265
0,lympho,148,18,4.0541,0.7155,0.9914,0.9483,1,1,0.9397,0.9569,0.9483,0.9655,0.9914
0,mnist,7603,100,9.2069,0.7747,0.8431,0.7246,0.5769,0.8033,0.8431,0.7101,0.9059,0.851,0.8497
0,musk,3062,166,3.1679,0.2716,1,0.6591,1,1,0.8247,0.6128,1,1,1
0,optdigits,5216,64,2.8758,0.4971,0.5922,0.4715,0.8553,0.7033,0.4029,0.4934,0.4041,0.4808,0.5016
0,pendigits,6870,16,2.2707,0.6957,0.9329,0.4277,0.9322,0.9756,0.7694,0.4056,0.8413,0.9376,0.9384


In [5]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.5,0.5,0.5714,0.5,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.209,0.5075,0.1194,0.3731,0.3433,0.2388,0.1493,0.3284,0.3731,0.4925
0,glass,214,9,4.2056,0,0,0.2,0,0,0,0.2,0,0,0
0,ionosphere,351,33,35.8974,0.8333,0.6111,0.7407,0.4259,0.6667,0.8704,0.7407,0.8704,0.7593,0.6481
0,letter,1600,32,6.25,0.3438,0.0625,0.4062,0.0312,0.0312,0.3438,0.3125,0.125,0.0938,0.0625
0,lympho,148,18,4.0541,0,0.5,0.5,1,1,0.5,0.5,0.5,0.5,0.5
0,mnist,7603,100,9.2069,0.384,0.365,0.3422,0.1217,0.2966,0.4183,0.3384,0.5133,0.3802,0.3688
0,musk,3062,166,3.1679,0.0714,1,0.2143,1,1,0.2857,0.2143,0.9762,1,1
0,optdigits,5216,64,2.8758,0.0299,0,0.0448,0.209,0.0299,0,0.0448,0,0,0
0,pendigits,6870,16,2.2707,0.1127,0.2394,0.0986,0.3099,0.3944,0.1549,0.0986,0.0986,0.3662,0.3803
